In [26]:
import paramiko
import os
import re

In [27]:
def run_in_master(command):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect("namenode", username="root", password="pass")
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(f"cd /app/ && . /env_var_path.sh && {command}")
    return (ssh_stdout.readlines(), ssh_stderr.readlines())

def merge_results(path):
    run_in_master(f"hdfs dfs -cat {path}/part-* | hdfs dfs -put - {path}/merged.txt")
    
def get_data_from_output_path(path):
    return f"{path}/merged.txt"

def print_hdfs_output(path):
    raw = run_in_master(f"hdfs dfs -cat {get_data_from_output_path(path)}")[0]
    print("\n".join(raw[0:1000]))

def hdfs_mkdir(path):
    run_in_master(f"hdfs dfs -mkdir -p /{path}/")

def hdfs_upload(path):
    directory = "/".join(path.split("/")[:-1])
    hdfs_mkdir(directory)
    cmd = f"hdfs dfs -put /data/master_volume/{path} /{directory}"
    print(cmd)
    code, output = run_in_master(cmd)
    print(f"exit code {code}")
    print(output)

In [28]:
def get_pig_time_from_results(res):
    m = re.search('Pig script completed in .*\((\d*) ms\)',res[1][-1])
    return float(m.group(1)) / 1000

In [ ]:
hdfs_upload("pig_scripts/elephant-bird-core-4.17.jar")
hdfs_upload("pig_scripts/elephant-bird-pig-4.17.jar")
hdfs_upload("pig_scripts/elephant-bird-hadoop-compat-4.17.jar")

In [30]:
covid_result = run_in_master("pig -x mapreduce /data/master_volume/pig_scripts/covid_01.pig")
covid_time = get_pig_time_from_results(covid_result)
covid_time

40.865

In [31]:
merge_results("/pig_output/covid_01")

In [32]:
print_hdfs_output("/pig_output/covid_01")

{"date":"NIU","location":"Oceania","total_cases":"Niue","new_cases":"2022-05-05","total_deaths":"9.0","new_deaths":"0.0","new_cases_per_million":0,"average_new_cases_per_million":10632.921378213763}

{"date":"NIU","location":"Oceania","total_cases":"Niue","new_cases":"2022-05-06","total_deaths":"9.0","new_deaths":"0.0","new_cases_per_million":0,"average_new_cases_per_million":10632.921378213763}

{"date":"NIU","location":"Oceania","total_cases":"Niue","new_cases":"2022-05-07","total_deaths":"9.0","new_deaths":"0.0","new_cases_per_million":0,"average_new_cases_per_million":10632.921378213763}

{"date":"NIU","location":"Oceania","total_cases":"Niue","new_cases":"2022-05-08","total_deaths":"9.0","new_deaths":"0.0","new_cases_per_million":0,"average_new_cases_per_million":10632.921378213763}

{"date":"NIU","location":"Oceania","total_cases":"Niue","new_cases":"2022-05-09","total_deaths":"9.0","new_deaths":"0.0","new_cases_per_million":0,"average_new_cases_per_million":10632.921378213763}



In [33]:
steam_result = run_in_master("pig -x mapreduce /data/master_volume/pig_scripts/steam_01_combine.pig")
steam_time = get_pig_time_from_results(steam_result)
steam_time

28.722

In [34]:
merge_results("/pig_output/steam_01_combine")

In [35]:
print_hdfs_output("/pig_output/steam_01_combine")

{"steam_appid":"10","name":"Counter-Strike","positive":196557,"negative":5070,"owners":"10,000,000 .. 20,000,000","ccu":12877,"date":"Nov 1, 2000"}

{"steam_appid":"20","name":"Team Fortress Classic","positive":5574,"negative":907,"owners":"5,000,000 .. 10,000,000","ccu":76,"date":"Apr 1, 1999"}

{"steam_appid":"30","name":"Day of Defeat","positive":5116,"negative":564,"owners":"5,000,000 .. 10,000,000","ccu":124,"date":"May 1, 2003"}

{"steam_appid":"40","name":"Deathmatch Classic","positive":1897,"negative":417,"owners":"5,000,000 .. 10,000,000","ccu":4,"date":"Jun 1, 2001"}

{"steam_appid":"50","name":"Half-Life: Opposing Force","positive":13900,"negative":683,"owners":"5,000,000 .. 10,000,000","ccu":82,"date":"Nov 1, 1999"}

{"steam_appid":"60","name":"Ricochet","positive":3738,"negative":849,"owners":"5,000,000 .. 10,000,000","ccu":6,"date":"Nov 1, 2000"}

{"steam_appid":"70","name":"Half-Life","positive":71502,"negative":2552,"owners":"5,000,000 .. 10,000,000","ccu":583,"date":"N